In [ ]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import glob
import seaborn as sns
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.merge import Concatenate
from keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import cv2
from keras import backend as K
from pathlib import Path
%matplotlib inline


data_path = Path('D:\\Data\\SMILE\\chest_xray')
test = data_path / 'test'
train = data_path / 'train'
val = data_path / 'val'

test_normal = test/ 'NORMAL'
test_pneumonia = test/ 'PNEUMONIA'

train_normal = train/ 'NORMAL'
train_pneumonia = train/ 'PNEUMONIA'

val_normal = val/ 'NORMAL'
val_pneumonia = val/ 'PNEUMONIA'



ex_img = train_normal/'IM-0115-0001.jpeg'
Leena = data_path/'Lenna.png'


def distinction1(img): #clahe girdsize가 크면 클수록 대비가 좋아지긴하나 부자연스러움
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize = (8,8)) 
    return clahe.apply(img)

def distinction2(img): # 히스토그램 평활화
    return cv2.equalizeHist(img)

def distinction3(img): # 고주파 영역 clahe 영향 키우고 저주파영역 he영향 키움 clahe 자연스럽게 만든거라 생각 하면됨
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize = (8,8)) 
    clahe_img = clahe.apply(img)
    he_img = cv2.equalizeHist(img)
    std = img.std()
    for dh in range(0,img.shape[0]-7,8):
        for dw in range(0,img.shape[1]-7,8):
            crop = img[dh:dh+8,dw:dw+8]
            if std >= crop.std(): #상대적으로 저주파 지역
                img[dh:dh+8,dw:dw+8] = clahe_img[dh:dh+8,dw:dw+8]
            else:
                img[dh:dh+8,dw:dw+8] = he_img[dh:dh+8,dw:dw+8]
    return img
        

img = cv2.imread(str(ex_img), cv2.IMREAD_UNCHANGED)  ## 우선은 기존 사진을 불러오는게 맞을듯
f_img = img.copy()
f_img = cv2.resize(f_img ,dsize=(0,0),fx=0.6,fy=0.6, interpolation = cv2.INTER_AREA)
cv2.imshow('img', f_img)

height ,width = f_img.shape
crow , ccol = int(height/2), int(width/2)
dh, dw = int(height/20), int(width/20)

img_zip = []
for i in range(1,11):
        
    background = np.zeros((height, width),dtype= int)
    f = np.fft.fft2(f_img)
    fshift = np.fft.fftshift(f)
    fshift2 = fshift.copy()
    if i == 1:
        fshift[crow-(dh*i):crow+(dh*i),ccol-(dw*i):ccol+(dw*i)] = 0
        fshift2 = fshift2 - fshift
    elif i ==10:
        fshift[crow-(dh*(i-1)):crow+(dh*(i-1)),ccol-(dh*(i-1)):ccol+(dh*(i-1))] = 0
        fshift2 = fshift2 - fshift
    else: 
        fshift[crow-dh*(i-1):crow+dh*(i-1),ccol-dw*(i-1):ccol+dw*(i-1)] = 0
        fshift2[crow-(dh*i):crow+(dh*i),ccol-(dw*i):ccol+(dw*i)] = 0
        fshift2 = fshift -  fshift2
    f_ishift = np.fft.ifftshift(fshift2)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)
    img_zip.append(img_back)
    
weight_img = np.zeros(f_img.shape, np.float32 )
print(weight_img.shape)
for i, _img in enumerate(img_zip):
    if i <3 :
        weight_img = weight_img + 1.2*_img
    elif (i>=3)and(i<7):
        weight_img = weight_img + 1.8*_img
    else:
        weight_img = weight_img + 0.6*_img
        
weight_img = weight_img.astype('uint8')
cv2.imshow('weight_img', weight_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

def img_resize1(img,w,h):  # padding을 통해 size 맞춤
    if((w/img.shape[1]) < h/img.shape[0]):
        percent = w/img.shape[1]
        img = cv2.resize(img,dsize=(0,0),fx=percent,fy=percent, interpolation = cv2.INTER_LINEAR)
        dh = (h-img.shape[0])/2
        M = np.array([[1,0,0],[0,1,dh]],dtype=np.float64)  
        img = cv2.warpAffine(img, M, (w, h))
    else:
        percent = h/img.shape[0]
        img = cv2.resize(img,dsize=(0,0),fx=percent,fy=percent, interpolation = cv2.INTER_LINEAR)
        dw = (w-img.shape[1])/2
        M = np.array([[1,0,dw],[0,1,0]],dtype=np.float64)  
        img = cv2.warpAffine(img, M, (w, h))
    return img
        
def img_resize2(img,w,h): #crop을 통해 size 맞춤
    if((w/img.shape[1]) > h/img.shape[0]):
        percent = w/img.shape[1]
        img = cv2.resize(img,dsize=(0,0),fx=percent,fy=percent, interpolation = cv2.INTER_LINEAR)
        print("1",img.shape[0])
        dh = (img.shape[0]-h)//2
        img = img[dh:dh+h,:].copy()
    else:
        percent = h/img.shape[0]
        img = cv2.resize(img,dsize=(0,0),fx=percent,fy=percent, interpolation = cv2.INTER_LINEAR)
        print("2",img.shape[1])
        dw = (img.shape[1]-w)//2
        img = img[:,dw:dw+w].copy()
    return img
    
def img_resize3(img,w,h):  # resize1과 동일 보간법 차이
    if((w/img.shape[1]) < h/img.shape[0]):
        percent = w/img.shape[1]
        img = cv2.resize(img,dsize=(0,0),fx=percent,fy=percent, interpolation = cv2.INTER_AREA)
        dh = (h-img.shape[0])/2
        M = np.array([[1,0,0],[0,1,dh]],dtype=np.float64)  
        img = cv2.warpAffine(img, M, (w, h))
    else:
        percent = h/img.shape[0]
        img = cv2.resize(img,dsize=(0,0),fx=percent,fy=percent, interpolation = cv2.INTER_AREA)
        dw = (w-img.shape[1])/2
        M = np.array([[1,0,dw],[0,1,0]],dtype=np.float64)  
        img = cv2.warpAffine(img, M, (w, h))
    return img
        
def img_resize4(img,w,h): #resize2과 동일 보간법 차이
    if((w/img.shape[1]) > h/img.shape[0]):
        percent = w/img.shape[1]
        img = cv2.resize(img,dsize=(0,0),fx=percent,fy=percent, interpolation = cv2.INTER_AREA)
        print("1",img.shape[0])
        dh = (img.shape[0]-h)//2
        img = img[dh:dh+h,:].copy()
    else:
        percent = h/img.shape[0]
        img = cv2.resize(img,dsize=(0,0),fx=percent,fy=percent, interpolation = cv2.INTER_AREA)
        print("2",img.shape[1])
        dw = (img.shape[1]-w)//2
        img = img[:,dw:dw+w].copy()
    return img

# test
f_img = img.copy()
cv2.imshow('before', f_img)
f_img1 = img_resize1(f_img,600,500)
f_img2 = img_resize3(f_img,600,500)
print(f_img2.shape)
cv2.imshow('after1', f_img1)
cv2.imshow('after2', f_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

f_img = img.copy()
f_img2 = img_resize2(f_img,600,500)
cv2.imshow('img', f_img2)
cv2.imshow('img1', distinction1(f_img2))
cv2.imshow('img2', distinction2(f_img2))
cv2.imshow('img3', distinction3(f_img2))
cv2.waitKey(0)
cv2.destroyAllWindows()

def norm1(img):
    img = (img-img.min())/(img.max()-img.min())
    return img

def norm2(img):
    img = (img - img.mean())/(img.std())
    return img

norm_cases = train_normal.glob('*.jpeg')
pneumonia_cases = train_pneumonia.glob('*.jpeg')

train_data = []
for img in norm_cases:
    train_data.append((img, 0))
    
for img in pneumonia_cases:
    train_data.append((img, 1))

train_data = pd.DataFrame(train_data, columns=['image', 'label'], index = None)
train_data = train_data.sample(frac=1.).reset_index(drop=True)
sns.countplot(x='label', data=train_data)

# 배운점 
    #os.listdir = 해당 디렉토리 파일명만 가져옴
    #glob = 해당 디렉토리의 경로명까지 가져옴 ->내생각엔 사진을 불러오는 느낌이라기 보단 경로를 불러오는 느낌
    #glob 은 generator를 반환함 
    #dstack, stack, concatenate 차이 

normal_cases = val_normal.glob('*.jpeg')
pneumonia_cases = val_pneumonia.glob('*.jpeg')

valid_data = []
valid_labels = []

for img in normal_cases:
    img = cv2.imread(str(img), cv2.IMREAD_UNCHANGED)
    img = distinction3(img) ## 선명도 높이기
    img = img_resize3(img,224,224) ## img resize
    img = norm1(img) ## 정규화 
    img = np.dstack([img])
    label = to_categorical(0, num_classes=2)
    valid_data.append(img)
    valid_labels.append(label)

for img in pneumonia_cases:
    img = cv2.imread(str(img), cv2.IMREAD_UNCHANGED)
    img = distinction3(img) ## 선명도 높이기
    img = img_resize3(img,224,224) ## img resize
    img = norm1(img) ## 정규화
    img = np.dstack([img])
    label = to_categorical(1, num_classes=2)
    valid_data.append(img)
    valid_labels.append(label)
    
# Convert the list into numpy arrays
valid_data = np.array(valid_data)
valid_labels = np.array(valid_labels)
print("image 2 ", valid_data[1].shape)

print("Total number of validation examples: ", valid_data.shape)
print("Total number of labels:", valid_labels.shape)


def data_gen(data, batch_size, img_size):
    n = len(data)
    steps = n//batch_size
    
    
    batch_data = np.zeros((batch_size,img_size[0],img_size[1],1), dtype=np.float32)
    batch_labels =np.zeros((batch_size,2), dtype=np.float32) ## 수정필요
    
    indices = np.arange(n)
    np.random.shuffle(indices)
    i = 0
    while True:
        count = 0
        next_batch = indices[(i*batch_size):(i+1)*batch_size]
        for j, idx in enumerate(next_batch):
            img_name = data.iloc[idx]['image']
            label = data.iloc[idx]['label']
            
            encoded_label = to_categorical(label, num_classes=2) ## 수정 필요
            
            img = cv2.imread(str(img_name), cv2.IMREAD_GRAYSCALE)
            img = distinction3(img) ## 선명도 높이기
            img = img_resize3(img,img_size[1],img_size[0]) ## img resize
            img = norm1(img) ## 정규화
            img = np.dstack([img]) ## 차원 맞추기
            batch_data[count] = img
            batch_labels[count] = encoded_label
            
            count +=1
            if count==batch_size -1:
                break
        i += 1
        yield batch_data, batch_labels
        
        if i>=steps:
            i=0
            np.random.shuffle(indices)